# 3: Example of a Minimum Viable Product

Here's an MVP solution that assesses a trading strategy for edge.

Fund managers report their holdings every month. They don’t want to tell investors that they lost money on the latest meme stock.
So they sell the meme stocks and buy higher-quality assets, like bonds.

You might be able to take advantage of this effect by buying bonds toward the end of the month and selling them at the beginning.

In [1]:
import pandas as pd
import numpy as np

from openbb_terminal.sdk import openbb

tlt = openbb.stocks.load(
    "TLT",
    start_date="2002-01-01",
    end_date="2022-06-30"
)

tlt["log_return"] = np.log(
    tlt['Adj Close'] / tlt['Adj Close'].shift(1)
)

tlt["day_of_month"] = tlt.index.day
tlt["year"] = tlt.index.year

grouped_by_day = tlt.groupby("day_of_month").log_return.mean()

grouped_by_day.plot.bar()

ModuleNotFoundError: No module named 'openbb_terminal'

Sure enough, there's evidence that returns are negative during the first few days and positive during the last few days.

Now build a naive strategy to test your hypothesis.

In [ ]:
tlt["first_week_returns"] = 0.0
wk_1 = tlt[tlt.day_of_month <= 7].log_return
tlt.loc[tlt.day_of_month <= 7, "first_week_returns"] = wk_1

tlt["last_week_returns"] = 0.0
wk_n = tlt[tlt.day_of_month >= 23].log_return
tlt.loc[tlt.day_of_month >= 23, "last_week_returns"] = wk_n

tlt["last_week_less_first_week"] = (
    tlt.last_week_returns - tlt.first_week_returns
)

(
    tlt
    .last_week_less_first_week
    .cumsum()
    .plot()
)

Now that you have some confidence that the strategy is worth exploring, invest the extra time to backtest it.